In [1]:
#Imports
import urllib3
import os
import requests
import re
from bs4 import BeautifulSoup
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Collect Links
links=[]
full_links=[]
StartYear=1990
EndYear=1991

while EndYear<2020:
    url = 'https://www.imdb.com/search/title/?title_type=feature&release_date='+str(StartYear)+
    '-01-01,'+str(EndYear)+'-01-01&num_votes=1000,&countries=us&languages=en&sort=num_votes,desc&count=50'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    for a in soup.find_all('a'): #, href=True):
        links.append(a.get('href'))

    links=['https://www.imdb.com'+a.strip() for a in links if a is not None and a.startswith('/title/tt')]
    full_links=full_links+links
    
    StartYear=StartYear+1
    EndYear=EndYear+1

In [3]:
filtered_links=[]

for k in np.arange(0, len(full_links)-1): #clean links
    full_links[k]=full_links[k].replace("vote?v=X;k=","")
    full_links[k]=full_links[k].replace("=","")
    
temp = full_links[0] #remove duplicates   
filtered_links.append(full_links[0])
for x in np.arange(1, len(full_links)-1):
    if (full_links[x]!=temp):
        filtered_links.append(full_links[x])
        print(full_links[x])
    temp=full_links[x]

https://www.imdb.com/title/tt0099785/
https://www.imdb.com/title/tt0099487/
https://www.imdb.com/title/tt0099088/
https://www.imdb.com/title/tt0099674/
https://www.imdb.com/title/tt0099423/
https://www.imdb.com/title/tt0100802/
https://www.imdb.com/title/tt0100405/
https://www.imdb.com/title/tt0099348/
https://www.imdb.com/title/tt0099653/
https://www.imdb.com/title/tt0100157/
https://www.imdb.com/title/tt0099810/
https://www.imdb.com/title/tt0100403/
https://www.imdb.com/title/tt0099938/
https://www.imdb.com/title/tt0100507/
https://www.imdb.com/title/tt0100150/
https://www.imdb.com/title/tt0099077/
https://www.imdb.com/title/tt0100814/
https://www.imdb.com/title/tt0099871/
https://www.imdb.com/title/tt0099700/
https://www.imdb.com/title/tt0100758/
https://www.imdb.com/title/tt0100935/
https://www.imdb.com/title/tt0100502/
https://www.imdb.com/title/tt0099582/
https://www.imdb.com/title/tt0099371/
https://www.imdb.com/title/tt0099052/
https://www.imdb.com/title/tt0099365/
https://www.

In [4]:
# Define DF for the data
column_list=['Rank','Movie_name' ,'URL' ,'Release_Year' ,'IMDB_Rating' ,'Metascore' ,
'Reviewer_count' ,'Censor_Board_Rating' ,'Movie_Length' ,'Genre_1' ,
'Genre_2' ,'Genre_3' ,'Genre_4' ,'Release_Date' ,'Story_Summary' ,
'Director' ,'Writer_1' ,'Writer_2' ,'Writer_3' ,'Star_1' ,
'Star_2' ,'Star_3' ,'Star_4' ,'Star_5' ,'Plot_Keywords' ,'Budget' ,
'Gross_USA' ,'Cum_Worldwide_Gross' ,'Production_Company' 
            ]
df = pd.DataFrame(columns=column_list)
df

,Rank,Movie_name,URL,Release_Year,IMDB_Rating,Metascore,Reviewer_count,Censor_Board_Rating,Movie_Length,Genre_1,...,Star_1,Star_2,Star_3,Star_4,Star_5,Plot_Keywords,Budget,Gross_USA,Cum_Worldwide_Gross,Production_Company


In [5]:
# Loop through URLs and retrive data to dataframe (May take a few minutes!)

for x in np.arange(0, len(filtered_links)-1):
    try:

    #---------------------------Load html page for each movie in movies list
        url=filtered_links[x]
        req = requests.get(url)
        page = req.text
        soup = BeautifulSoup(page, 'html.parser')
    
    #---------------------------Retrieve Movie details from html page
        
        Movie_name=(soup.find("div",{"class":"title_wrapper"}).get_text(strip=True).split('|')[0]).split('(')[0]
        
        year_released=((soup.find("div",{"class":"title_wrapper"}).get_text(strip=True).split('|')[0]).split('(')[1]).split(')')[0]
        
        imdb_rating=soup.find("span",{"itemprop":"ratingValue"}).text
        
        metascore=soup.find("div",{"class":"titleReviewBarItem"}).get_text(strip=True).replace("MetascoreFrommetacritic.com","")
    
        reviewer_count=soup.find("span",{"itemprop":"ratingCount"}).text
    
        subtext= soup.find("div",{"class":"subtext"}).get_text(strip=True).split('|') #Censor_rating
        if len(subtext)<4:
            censor_rating='Not Rated'
            movie_len=subtext[0] #Length
            genre_list=subtext[1].split(',') #Genres
            while len(genre_list)<4:         genre_list.append(" ")
            genre_1,genre_2,genre_3,genre_4=genre_list
            release_date=subtext[2] #Release Date
        else:
            censor_rating=subtext[0]
            movie_len=subtext[1]
            genre_list=subtext[2].split(',')
            while len(genre_list)<4:         genre_list.append(" ")
            genre_1,genre_2,genre_3,genre_4=genre_list
            release_date=subtext[3]
        
        #Summary
        story_summary=soup.find("div",{"class":"summary_text"}).get_text(strip=True).strip()
        
        #---------------------------Director,Writer and Actor details
        b=[]
        for a in soup.find_all("div",{"class":"credit_summary_item"}):
            c=re.split(',|:|\|',a.get_text(strip=True)) 
            b.append(c)
        stars=b.pop()
        writers=b.pop()
        directors=b.pop()
        if 'See full cast & crew»' in stars: stars.remove('See full cast & crew»')
        if '1 more credit»' in writers: writers.remove('1 more credit»') 
        if '1 more credit»' in directors: directors.remove('1 more credit»')
        stars=stars[1:]
        writers=writers[1:]
        directors=directors[1:]
        while len(stars)<5:         stars.append(" ")
        while len(writers)<3:         writers.append(" ")
    
        star_1,star_2,star_3,star_4,star_5=stars
        
        writer_1=writers[0]
        writer_2=writers[1]
        writer_3=writers[2]
        
        director=directors[0]
        
        #---------------------------Plot Keywords
        b=[]
        for a in soup.find_all("span",{"class":"itemprop"}):     b.append(a.get_text(strip=True))  
        
        plot_keywords='|'.join(b)
        
        #---------------------------Commercial details and Prod Company
        
        
        b=[]                    #---------------------------Remove unwanted entries
        d={'Budget':'', 'Opening Weekend USA':'','Gross USA':'','Cumulative Worldwide Gross':'','Production Co':''}
        for a in soup.find_all("div",{"class":"txt-block"}):
            c=a.get_text(strip=True).split(':')
            if c[0] in d:
                b.append(c)
    
        for i in b:             #---------------------------Update default values if entries are found
                if i[0] in d: 
                    d.update({i[0]:i[1]})                
    
        production_company=d['Production Co'].split('See more')[0]
        cum_world_gross=d['Cumulative Worldwide Gross'].split(' ')[0]
        gross_usa=d['Gross USA'].split(' ')[0]
        budget=d['Budget']
        
        #---------------------------Dictionary to holds all details
        movie_dict={
            'Rank':x+1,
            'Movie_name' : Movie_name,
            'URL' : url,
            'Release_Year' : year_released,
            'IMDB_Rating' : imdb_rating,
            'Metascore' : metascore,
            'Reviewer_count' : reviewer_count,
            'Censor_Board_Rating' : censor_rating,
            'Movie_Length' : movie_len,
            'Genre_1' : genre_1,
            'Genre_2' : genre_2,
            'Genre_3' : genre_3,
            'Genre_4' : genre_4,
            'Release_Date' : release_date,
            'Story_Summary' : story_summary,
            'Director' : director,
            'Writer_1' : writer_1,
            'Writer_2' : writer_2,
            'Writer_3' : writer_3,
            'Star_1' : star_1,
            'Star_2' : star_2,
            'Star_3' : star_3,
            'Star_4' : star_4,
            'Star_5' : star_5,
            'Plot_Keywords' : plot_keywords,
            'Budget' : budget,
            'Gross_USA' : gross_usa,
            'Cum_Worldwide_Gross' : cum_world_gross,
            'Production_Company' : production_company
            }
        #---------------------------Append rows to dataframes using dictionary
        df = df.append(pd.DataFrame.from_records([movie_dict],columns=movie_dict.keys() ) )
    
    except Exception:
        pass

In [6]:
df=df[column_list]  
df=df.set_index(['Rank'], drop=False) #should be run only once

In [7]:
df

,Rank,Movie_name,URL,Release_Year,IMDB_Rating,Metascore,Reviewer_count,Censor_Board_Rating,Movie_Length,Genre_1,...,Star_1,Star_2,Star_3,Star_4,Star_5,Plot_Keywords,Budget,Gross_USA,Cum_Worldwide_Gross,Production_Company
Rank,,,,,,,,,,,,,,,,,,,,,
1,1,Goodfellas,https://www.imdb.com/title/tt0099685/,1990,8.7,90,"983,254",16,2h 26min,Biography,...,Robert De Niro,Ray Liotta,Joe Pesci,,,gangster|organized crime|sadistic psychopath|m...,"$25,000,000(estimated)","$46,836,214","$46,879,633",Warner Bros.
2,2,Home Alone,https://www.imdb.com/title/tt0099785/,1990,7.6,63,"448,940",Not Rated,1h 43min,Comedy,...,Macaulay Culkin,Joe Pesci,Daniel Stern,,,christmas|home alone|burglary|child protagonis...,"$18,000,000(estimated)","$285,761,243","$477,063,114","Hughes Entertainment,Twentieth Century Fox"
3,3,Edward Scissorhands,https://www.imdb.com/title/tt0099487/,1990,7.9,74,"435,025",PG,1h 45min,Drama,...,Johnny Depp,Winona Ryder,Dianne Wiest,,,scissors|compassion|surrealism|waterbed|love,"$20,000,000(estimated)","$56,362,352","$86,024,005",Twentieth Century Fox
4,4,Back to the Future Part III,https://www.imdb.com/title/tt0099088/,1990,7.4,55,"389,988",Not Rated,1h 58min,Adventure,...,Michael J. Fox,Christopher Lloyd,Mary Steenburgen,,,year 1955|year 1885|woman|purple dress|girl ne...,"$40,000,000(estimated)","$88,277,583","$249,130,916","Universal Pictures,Amblin Entertainment,U-Driv..."
5,5,The Godfather: Part III,https://www.imdb.com/title/tt0099674/,1990,7.6,60,"347,836",PG,2h 42min,Crime,...,Al Pacino,Diane Keaton,Andy Garcia,,,opera|vatican|italy|gangster|catholic church,"$54,000,000(estimated)","$66,666,062","$136,766,062","Paramount Pictures,Zoetrope Studios"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,1456,Maze Runner: The Death Cure,https://www.imdb.com/title/tt4500922/,2018,6.2,50,"105,748",PG-13,2h 23min,Action,...,Dylan O'Brien,Ki Hong Lee,Kaya Scodelario,,,human experimentation|virus|post apocalypse|re...,"$62,000,000(estimated)","$58,032,443","$288,175,335","Gotham Group,Temple Hill Entertainment,Twentie..."
1457,1457,Pacific Rim: Uprising,https://www.imdb.com/title/tt2557478/,2018,5.6,44,"101,700",PG-13,1h 51min,Action,...,John Boyega,Scott Eastwood,Cailee Spaeny,,,sea doo|woman with masculine short hair|woman ...,"$150,000,000(estimated)","$59,874,525","$290,930,148","Legendary Entertainment,Universal Pictures,Leg..."
1458,1458,Skyscraper,https://www.imdb.com/title/tt5758778/,2018,5.8,51,"100,372",PG-13,1h 42min,Action,...,Dwayne Johnson,Neve Campbell,Chin Han,,,fire|skyscraper|terrorism|evil man|prosthetic leg,"$125,000,000(estimated)","$68,420,120","$304,868,961","Legendary Entertainment,Flynn Picture Company,..."


In [8]:
df.to_csv('IMDB_ManyRatings_Dataset.csv', index=False)